In [7]:
import numpy as np
import pandas as pd
from google.cloud import bigquery
import re
import datetime
from datetime import datetime, timedelta
from tabulate import tabulate

In [2]:
rundate = datetime.strftime(datetime.now() + timedelta(hours=7) - timedelta(days=1), '%Y%m%d')
rundate = '20221116'

In [3]:
client = bigquery.Client()

In [4]:
# SAP ssale đã thay đổi lấy theo bảng fact mới tự phát triển
sap_sales_trans = (
    client.query('''
        SELECT  
        AOFY
        ,case when MANDT = '912' then concat(MANDT,KUNNR)
        else KUNNR end KUNNR
        ,STATUS
        ,CALENDAR_DATE
        ,RESV_SIGN_DATE
        ,CONTRACT_SIGN_DATE
        ,CANCEL_DATE
        ,TRANSFER_DATE
        ,REV_TRANSFER_DATE
        ,REVENUE_SALE_BF_DISCOUNT	
        from `vhm-dataops-prod.VH_DWH.FACT_VHM_SAP_TRANS`
    ''')
    .result()
    .to_dataframe()
)

In [5]:
len(sap_sales_trans)

461672

In [6]:
sap_sales_trans.head(5)

,AOFY,KUNNR,STATUS,CALENDAR_DATE,RESV_SIGN_DATE,CONTRACT_SIGN_DATE,CANCEL_DATE,TRANSFER_DATE,REV_TRANSFER_DATE,REVENUE_SALE_BF_DISCOUNT
0,Z/000-401-001.0001.1.811,0020025399,Huỷ HĐMB,20190221,20180809,20180818,20190221,00000000,00000000,1.035100e+09
1,Z/001-302-008.0001.1.120,0020036544,Huỷ HĐMB,20190301,20190124,20190214,20190301,00000000,00000000,1.272694e+10
2,Z/000-502-009.0003.1.119,0020028125,Huỷ HĐMB,20190506,20190124,20190124,20190506,00000000,00000000,2.823589e+09
3,Z/000-502-001.0001.1.906,0020032032,Huỷ HĐMB,20190509,20181103,20190110,20190509,00000000,00000000,2.072636e+09
4,Z/000-502-009.0003.1.120,0020028125,Huỷ HĐMB,20190509,20190124,20190124,20190509,00000000,00000000,2.781184e+09


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [ ]:
# SAP leasing
sap_lease_trans = (
    client.query('''
    SELECT
        PLANTNAME AS PROJECT,
        UNITCODE AS HOUSE_UNIT,
        ACCOUNTINGCODE AS sap_isu_id,
        VALIDFR AS TIME_START,
        VALIDTO AS TIME_END,
        PREMISETYPE AS PROPERTY_TYPE,
        UPDATE_DATE,
        AMOUNT,
        ADDRESS,
        BIRTHDATE,
        "LEASE" AS CUSTOMER_TYPE,
    FROM `vhm-dataops-prod.VH_RAWDATA.SAPAPI_LEASING_CONTRACT` 
    WHERE STATUS IS NOT NULL
        AND STATUS NOT IN ("Cancel", "Booking")
    ''')
    .result()
    .to_dataframe()
)

In [ ]:
# SAP Residence
sap_res_trans = (
    client.query('''
    SELECT
        PLANTNAME AS PROJECT,
        UNITCODE AS HOUSE_UNIT,
        ACCOUNTINGCODE AS sap_isu_id,
        case when cast(REGEXP_REPLACE(VALIDFR,'-','') as int) < 19950101 then UPDATE_DATE
        else VALIDFR end AS TIME_START,
        UPDATE_DATE,
        VALIDTO AS TIME_END,
        SMASTER,
        PREMISETYPE AS PROPERTY_TYPE,
        ADDRESS,
        REGEXP_REPLACE(BIRTHDATE,'-','') DOB,
        case when GENDER = 'Mr.' then 'male'
            when GENDER = 'Ms.' then 'female'
            when GENDER = 'Comp' then 'comp'
        end GENDER
    FROM `vhm-dataops-prod.VH_RAWDATA.SAPAPI_RESIDENT`
    where SMASTER is not null or SMASTER <> ''
    ''')
    .result()
    .to_dataframe()
)